In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from IPython.display import display
from sklearn import metrics
from sklearn.feature_selection import RFE, RFECV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from tools import (COLUMNS_QUANT, COLUMNS_CAT, datasets)
from kneed import KneeLocator
import json
from scipy.sparse import csr_matrix
pd.options.display.max_rows = 80

f3_scorer = metrics.make_scorer(metrics.fbeta_score, beta=3)
f5_scorer = metrics.make_scorer(metrics.fbeta_score, beta=5)

t = time.time()

In [ ]:
df = pd.read_csv('data/df_train_prepro.csv').sample(frac=1)
df.shape

## 1. Régression logistique

Pour la régression logistique, on enlève les premières colonnes pour les variables indicatrices.

In [ ]:
datasets_df = datasets(df, drop='first')
X_quant_scaled = datasets_df['X_quant_scaled']
X_only_quant_scaled = datasets_df['X_only_quant_scaled']
y = datasets_df['y']

### 1.1. Méthode RFE

In [ ]:
def run_RFE(model, n_features_to_select, X, y, export=None):
    # create the RFE model and select 3 attributes
    rfe = RFE(model, n_features_to_select=n_features_to_select)
    X_sparse = csr_matrix(X)
    rfe = rfe.fit(X_sparse, y)

    # summarize the selection of the attributes
    #  print(rfe.support_)
    #  print(rfe.ranking_)
    
    dico = {"Rank": rfe.ranking_,
            "Support": rfe.support_}
    rf_df = pd.DataFrame(dico, index=X.columns).sort_values(by='Rank', ascending=True)

    display(rf_df)
    
    support = rf_df[rf_df.Support == True].index.to_list()
    
    columns_quant = sorted(list(set(support).intersection(set(COLUMNS_QUANT))))    
    columns_cat = sorted(list(set(support).difference(set(COLUMNS_QUANT))))
    
    print("Columns_quant :")
    display(columns_quant)
    print("\nColumns_cat :")
    display(columns_cat)    
    
    if export:        
        dico_export = {'columns_quant': columns_quant,
                   'columns_cat': columns_cat}
        with open(f"backups/{export}.json", 'w') as f:
            json.dump(dico_export, f)
        print(f"\nExportation : backups/{export}.json")

In [ ]:
%%time
model = LogisticRegression()
run_RFE(model, 8, X_quant_scaled, y)

In [ ]:
%%time
model = LogisticRegression()
run_RFE(model, 3, X_quant_scaled, y)

Pour ne pas avoir à définir un nombre de features, nous allons utiliser une méthode de Cross Validation qui permettra de calculer le nombre de features optimal.

### 1.2. Méthode RFECV

In [ ]:
def run_RFECV(model, X, y, score=f3_scorer, min_features_to_select=1, n_jobs=-1, knee=False):
    rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(5),
                  scoring=score,
                  min_features_to_select=min_features_to_select,
                  n_jobs=n_jobs)
    X_sparse = csr_matrix(X)
    rfecv.fit(X_sparse, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Nombre de variables sélectionnées")
    if score == f3_scorer:
        plt.ylabel(f"Score F3 moyen")
    else:
        plt.ylabel(f"Score {str(score)} moyen")
    x = range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select)
    y = rfecv.grid_scores_
    plt.plot(x, y)
    if type(knee) == bool and knee:
        kneedle = KneeLocator(x, y)
        plt.vlines(kneedle.knee, plt.ylim()[0], plt.ylim()[1], linestyles="--", color='black', label="Coude")
    if type(knee) == int:
        plt.vlines(knee, plt.ylim()[0], plt.ylim()[1], linestyles="--", color='black', label="Coude")
    plt.show()

    if type(knee) == bool and knee:
        print(f"Nombre optimal de variables à sélectionner (coude) : {kneedle.knee}")
        return kneedle.knee
    if type(knee) == int:
        print(f"Nombre optimal de variables à sélectionner (coude) : {knee}")
        return knee

#### 1.2.1. Variables quantitatives

In [ ]:
%%time
model = LogisticRegression()
knee = run_RFECV(model, X_quant_scaled, y, score=f3_scorer, knee=True)

Le nombre de features optimal semble être 4. Nous allons donc maintenant calculer une RFE avec cette valeur pour déterminer les features à conserver.

In [ ]:
%%time
model = LogisticRegression()
run_RFE(model, 4, X_quant_scaled, y)

#### 1.2.2. Variables quantitatives + catégorielles

Rajoutons à présent les variables catégorielles.

In [ ]:
%%time
model = LogisticRegression()
knee = run_RFECV(model, X_only_quant_scaled, y, score=f3_scorer, knee=11)

Le nombre de features optimal semble être 11. Nous allons donc maintenant calculer une RFE avec cette valeur pour déterminer les features à conserver.

In [ ]:
%%time
model = LogisticRegression()
run_RFE(model, 11, X_only_quant_scaled, y, export='RFECV_LR')

## 2. Random Forest

Pour les modèles de type arbre, il n'est pas nécessaire d'enlever toutes les premières colonnes des variables indicatrices.

In [ ]:
datasets_df = datasets(df, drop='if_binary')
X = datasets_df['X']
y = datasets_df['y']

In [ ]:
%%time
model = RandomForestClassifier(n_jobs=-1)
knee = run_RFECV(model, X, y, score=f3_scorer, n_jobs=1, knee=9)

Le nombre de features optimal semble être 9. Nous allons donc maintenant calculer une RFE avec cette valeur pour déterminer les features à conserver.

In [ ]:
%%time
model = RandomForestClassifier(n_jobs=-1)
run_RFE(model, 9, X, y, export='RFECV_Forest')

## 3. XGBoost

In [ ]:
%%time
model = XGBClassifier(n_jobs=-1)
knee = run_RFECV(model, X, y, score=f3_scorer, n_jobs=1, knee=True)

In [ ]:
%%time
model = XGBClassifier(n_jobs=-1)
run_RFE(model, 8, X, y, export='RFECV_XGBoost')

In [ ]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")